In [1]:
from pathlib import Path
import base64

image_file_path = Path("ideas/image_assets/im1.png")

# Read the image into a base64 string
with open(image_file_path, "rb") as image_file:
    base64_img_string = f"data:image/png;base64,{base64.b64encode(image_file.read()).decode('utf-8')}"

In [2]:
import anywidget
from traitlets import Unicode, Int, Tuple, observe, Bool, List, Any, Dict
from pathlib import Path
import pathlib
import base64

class TldrawSetImage(anywidget.AnyWidget):
    @staticmethod
    def base64_to_image_dimensions(base64_img_string):
        base64_img_string_only = base64_img_string.split(",")[1]
        decoded_bytes = base64.b64decode(base64_img_string_only)
        if decoded_bytes[:8] != b"\x89PNG\r\n\x1a\n":
            raise ValueError("Invalid PNG file")
        ihdr_start = 8
        ihdr_end = decoded_bytes.find(b"IHDR") + 4 + 8
        ihdr_chunk = decoded_bytes[ihdr_start:ihdr_end]
        image_width = int.from_bytes(ihdr_chunk[8:12], byteorder="big")
        image_height = int.from_bytes(ihdr_chunk[12:16], byteorder="big")
        return image_width, image_height

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def set_image(self, base64img):
        if not base64img:
            raise ValueError("No image provided")

        image_width, image_height = self.base64_to_image_dimensions(base64img)
        self.image_dimensions = (int(image_width / 2), int(image_height / 2))
        self.base64img = base64img

    base64img = Unicode("").tag(sync=True)
    image_dimensions = Tuple(Int(), Int(), default_value=(0, 0)).tag(sync=True)

    
    length = Int(100).tag(sync=True)
    coord = List().tag(sync=True)

    path_root = pathlib.Path().home()
    _esm = path_root / "projects/jupyter-tldraw/src/tldraw/static" / "image_set.js"
    _css = path_root / "projects/jupyter-tldraw/src/tldraw/static" / "image_set.css"

In [3]:
widget = TldrawSetImage()
widget

TldrawSetImage(image_dimensions=(0, 0))

In [4]:
import ipywidgets as widgets
from IPython.display import display
output = widgets.Output()
def on_color_change(change):
    output.clear_output(wait=True)
    with output:
        print(widget.coord)
        widget.set_image(base64img = base64_img_string)

widget.observe(on_color_change, names='coord')
display(output)


widget.set_image(base64img = base64_img_string)

Output()